<a href="https://colab.research.google.com/github/jbian92/Convolutional-Neural-Networks/blob/main/Cat_vs_Dog_Image_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the image dataset

In [9]:
! wget https://www.ocf.berkeley.edu/~mingjie/tr/mnist/animals.zip
! unzip animals.zip
! rm animals.zip

Streaming output truncated to the last 5000 lines.
  inflating: cat/pixabay_cat_004546.jpg  
  inflating: cat/pixabay_cat_004547.jpg  
  inflating: cat/pixabay_cat_004549.jpg  
  inflating: cat/pixabay_cat_004550.jpg  
  inflating: cat/pixabay_cat_004551.jpg  
  inflating: cat/pixabay_cat_004552.jpg  
  inflating: cat/pixabay_cat_004553.jpg  
  inflating: cat/pixabay_cat_004554.jpg  
  inflating: cat/pixabay_cat_004555.jpg  
  inflating: cat/pixabay_cat_004556.jpg  
  inflating: cat/pixabay_cat_004557.jpg  
  inflating: cat/pixabay_cat_004558.jpg  
  inflating: cat/pixabay_cat_004559.jpg  
  inflating: cat/pixabay_cat_004560.jpg  
  inflating: cat/pixabay_cat_004561.jpg  
  inflating: cat/pixabay_cat_004563.jpg  
  inflating: cat/pixabay_cat_004564.jpg  
  inflating: cat/pixabay_cat_004565.jpg  
  inflating: cat/pixabay_cat_004566.jpg  
  inflating: cat/pixabay_cat_004567.jpg  
  inflating: cat/pixabay_cat_004568.jpg  
  inflating: cat/pixabay_cat_004569.jpg  
  inflating: cat/pixabay_

Plan: 
1. Convert images from the dataset into black and white 
2. Create a model that can predict if a black and white image is of a cat or dog

In [2]:
from sklearn.model_selection import train_test_split                    # used to split data into a training dataset and testing dataset
from tensorflow import keras                                            # used to create machine learning models and train them
from keras.models import Sequential                                     # used to create a base sequential model because CNN is a sequential model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D  # used to create CNN architecture
from PIL import Image                                                   # used to process images
import numpy as np

In [11]:
images = []
labels = []

for i in range(1,4900):
  catpath = f'cat/flickr_cat_{i:06}.jpg'
  dogpath = f'dog/flickr_dog_{i:06}.jpg'

  try:
    with Image.open(catpath) as im_cat:     # opens the image
      im_cat = im_cat.convert("L")          # converts the image to be black and white
      catpix = np.array(im_cat.getdata())
      catpix = catpix.reshape(512, 512, 1)
      images.append(catpix)                 # image gets added onto images 
      labels.append(np.array([1,0]))        # one hot encoded version of the label gets added onto labels
  except:
    continue

  try:
    with Image.open(dogpath) as im_dog:
      im_dog = im_dog.convert("L")
      dogpix = np.array(im_dog.getdata())
      dogpix = dogpix.reshape(512, 512, 1)
      images.append(dogpix)
      labels.append(np.array([0,1]))
  except:
    continue

  x = np.stack(images)
  y = np.stack(labels)

Define constants

In [4]:
batch_size = 16                # how many images get sent to the model at a time when training
num_classes = 2                # number of total classifications (cat or dog)
epochs = 10                    # number of times the model will see the entire training dataset

Create training and testing sets

In [12]:
# X_train represents training evidence
# X_test represents testing evidence
# y_train represents training labels
# y_test represents testing labels
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

Set up the model

In [13]:
model = Sequential()

# add convolution layers, pooling layers, and dropout layers with fully connected layers at the end
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(512,512,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(num_classes, activation='softmax'))

Train the model

In [14]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/10
61/61 [==============================] - 143s 2s/step - loss: 11.7020 - accuracy: 0.5354 - val_loss: 0.6718 - val_accuracy: 0.5288
Epoch 2/10
61/61 [==============================] - 92s 2s/step - loss: 0.8282 - accuracy: 0.5615 - val_loss: 0.6767 - val_accuracy: 0.6827
Epoch 3/10
61/61 [==============================] - 92s 2s/step - loss: 0.7309 - accuracy: 0.6145 - val_loss: 0.6890 - val_accuracy: 0.5697
Epoch 4/10
61/61 [==============================] - 92s 2s/step - loss: 0.6575 - accuracy: 0.6440 - val_loss: 0.5918 - val_accuracy: 0.6587
Epoch 5/10
61/61 [==============================] - 92s 2s/step - loss: 0.6651 - accuracy: 0.6573 - val_loss: 0.5770 - val_accuracy: 0.6731
Epoch 6/10
61/61 [==============================] - 92s 2s/step - loss: 0.6116 - accuracy: 0.7253 - val_loss: 1.0513 - val_accuracy: 0.6490
Epoch 7/10
61/61 [==============================] - 96s 2s/step - loss: 0.6677 - accuracy: 0.7291 - val_loss: 0.5109 - val_accuracy: 0.7981
Epoch 8/10
61/61 [

Test the model

In [15]:
model.evaluate(X_test, y_test, verbose=2)

13/13 - 6s - loss: 0.4254 - accuracy: 0.8341


[0.42541536688804626, 0.8341346383094788]